<a name="top" id="top"></a>

<div align="center">
    <h1>Sensor Placement and Fault Diagnosis in Real Water Distribution Netwroks</h1>
    <a href="https://github.com/anurag-r20">Anurag Ramesh</a>
    <br>
    <i>Graduate Research Assistant</i>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <br>
    <a href="https://github.com/bernalde">David E. Bernal Neira</a>
    <br>
    <i> Assistant Professor </i>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <br>
   <a href="https://colab.research.google.com/github.com/anurag-r20/WDN_Sensor_Placement/WDN_Sensor_Placement.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
    <a href="https://secquoia.github.io/">
        <img src="https://img.shields.io/badge/🌲⚛️🌐-SECQUOIA-blue" alt="SECQUOIA"/>
    </a>
</div>

## Sensor Placement in Water Distribution Networks (WDN)

In this notebook, we formulate an optimization problem to identify the optimal placement of pressure sensors in a Water Distribution Network (WDN). The WDN is represented on a constrained graph $G(V,E)$, where $V$ is the set of vertices and $E$ is the set of edges.

We consider two different formulations of the optimization problem, including Mixed Integer Programming (MIP), Mixed Integer Quadratic Programming (MIQP), and Quadratic Unconstrained Binary Optimization (QUBO). The optimization problem is solved using three different solvers: Gurobi, Simulated Annealing, and D-Wave's implementation of Quantum Annealing via **[neal](https://github.com/dwavesystems/dwave-neal)**.

Additionally, we leverage D-Wave's package **[dwavebinarycsp](https://github.com/dwavesystems/dwavebinarycsp)** to translate constraint satisfaction problems into QUBOs. For Groebner basis computations, we use **[Sympy](https://www.sympy.org/)** for symbolic computation in Python and **[Networkx](https://networkx.github.io/)** for network models and graphs.

In [ ]:
# If using this on Google collab, we need to install the packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Let's install dimod, neal, and pyomo
if IN_COLAB:
    !pip install -q pyomo
    !pip install dimod
    !pip install dwave-neal

if IN_COLAB:
    !pip install dwave-ocean-sdk

In [ ]:
# Import the Pyomo library, which can be installed via pip, conda or from Github https://github.com/Pyomo/pyomo
import pyomo.environ as pyo
# Import the Dwave packages dimod and neal
import dimod
import neal
from tabu import TabuSampler
# Import Matplotlib to generate plots
import matplotlib.pyplot as plt
# Import numpy and scipy for certain numerical calculations below
import numpy as np
from scipy.special import gamma
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import os
import time
import operator
from collections import OrderedDict
from collections import Counter
# Import networkx for graph problems
import networkx as nx
from matplotlib.lines import Line2D


In [ ]:
def WDN_network_data(city, base_dir=None):

    """Fetches and processes Water Distribution Network (WDN) data for a given city.

    This function reads node, edge, and water consumption data from specified files
    for a given city and returns the data as dictionaries.

    Args:
        city (str): The name of the city to fetch data for. Should be one of 'Apulia', 
            'Fossolo', or 'Test'.
        base_dir (str, optional): The base directory where the data files are located. 
            Defaults to the directory of the script.

    Returns:
        tuple: A tuple containing three dictionaries:
            - nodes (dict): Dictionary of nodes with node names as keys and (x, y) 
              coordinates as values.
            - edges (dict): Dictionary of edges with tuples of node pairs as keys.
            - water_consumption (dict): Dictionary of water consumption with node names
              as keys and consumption values as values.

    Raises:
        FileNotFoundError: If any of the data files do not exist.
        ValueError: If the provided city name is not valid.

    Examples:
        >>> nodes, edges, water_consumption = WDN_network_data('Apulian')
        >>> print(nodes)
        {'J1': (10.0, 20.0), 'J2': (15.0, 25.0)}
    """

    # Set base directory
    if base_dir is None:
        base_dir = os.path.dirname(os.path.abspath(__file__))  # Set to script's directory
    
    data_dir = os.path.join(base_dir, 'Data/WDN_Data')
    city_files = {
        "Apulia": {
            "nodes": f"{data_dir}/Apulia_WDN/Apulia_WDN_Nodes.txt",
            "edges": f"{data_dir}/Apulia_WDN/Apulia_WDN_Edges.txt",
            "water_consumption": f"{data_dir}/Apulia_WDN/Apulia_WDN_Water_Consumption.txt"
        },
        "Fossolo": {
            "nodes": f"{data_dir}/Fossolo_WDN/Fossolo_WDN_Nodes.txt",
            "edges": f"{data_dir}/Fossolo_WDN/Fossolo_WDN_Edges.txt",
            "water_consumption": f"{data_dir}/Fossolo_WDN/Fossolo_WDN_Water_Consumption.txt"
        },
        "Test": {
            "nodes": f"{data_dir}/Test_WDN/Test_Nodes.txt",
            "edges": f"{data_dir}/Test_WDN/Test_Edges.txt",
            "water_consumption": f"{data_dir}/Test_WDN/Test_Water_Consumption.txt"
        },
        "ZJ": {
            "nodes": f"{data_dir}/ZJ_WDN/ZJ_WDN_Nodes.txt",
            "edges": f"{data_dir}/ZJ_WDN/ZJ_WDN_Edges.txt",
            "water_consumption": f"{data_dir}/ZJ_WDN/ZJ_WDN_Water_Consumption.txt"
        },
        "Modena": {
            "nodes": f"{data_dir}/Modena_WDN/Modena_WDN_Nodes.txt",
            "edges": f"{data_dir}/Modena_WDN/Modena_WDN_Edges.txt",
            "water_consumption": f"{data_dir}/Modena_WDN/Modena_WDN_Water_Consumption.txt"
        },
        "Kentucky": {
            "nodes": f"{data_dir}/Kentucky_WDN/Kentucky_WDN_Nodes.txt",
            "edges": f"{data_dir}/Kentucky_WDN/Kentucky_WDN_Edges.txt",
            "water_consumption": f"{data_dir}/Kentucky_WDN/Kentucky_WDN_Water_Consumption.txt"
        }
        #  "Test": {
        #     "nodes": f"{data_dir}/Test_WDN/Test_Nodes_5.txt",
        #     "edges": f"{data_dir}/Test_WDN/Test_Edges_5.txt",
        #     "water_consumption": f"{data_dir}/Test_WDN/Test_Water_Consumption_5.txt"
        # }      
    }

    # Check if the city is valid
    if city not in city_files:
        print(f"Error: Unknown city {city}.")
        return None, None, None

    # Get file paths
    nodes_file_path = city_files[city]["nodes"]
    edges_file_path = city_files[city]["edges"]
    water_consumption_file_path = city_files[city]["water_consumption"]

    # Print absolute file paths for debugging
    print(f"Nodes file path: {nodes_file_path}")
    print(f"Edges file path: {edges_file_path}")
    print(f"Water consumption file path: {water_consumption_file_path}")

    # Check if files exist
    for file_path in [nodes_file_path, edges_file_path, water_consumption_file_path]:
        if not os.path.exists(file_path):
            print(f"Error: The file {file_path} does not exist.")
            return None, None, None

    # Importing node data
    nodes_df = pd.read_csv(nodes_file_path, skiprows=1, names=["Node", "X", "Y"]) # nodes dataframe (exclude heading)
    nodes_df['Node'] = nodes_df['Node'].str.strip() # strip spaces
    nodes = {row['Node']: (float(row['X']), float(row['Y'])) for i, row in nodes_df.iterrows()} # Convert the data to a dictionary with node names as keys and (x, y) tuples as values.

    # Importing edge data
    edges_df = pd.read_csv(edges_file_path, skiprows=1, names=["Node1", "Node2"]) # edges dataframe
    edges_df['Node1'] = edges_df['Node1'].str.strip() # strip spaces
    edges_df['Node2'] = edges_df['Node2'].str.strip()
    edges = {((row['Node1']), (row['Node2'])) for i, row in edges_df.iterrows()} # Convert the data to a dictionary with node names as keys and (x, y) tuples as values.
    
    # Importing water consumption data
    water_consumption_df = pd.read_csv(water_consumption_file_path, skiprows=1, names=["Node", "Consumption"]) # water consumption dataframe
    water_consumption_df['Node'] = water_consumption_df['Node'].str.strip() # strip spaces
    water_consumption = {row['Node']: (float(row['Consumption'])) for i, row in water_consumption_df.iterrows()} # Convert the data to a dictionary with node names as keys and (x, y) tuples as values.

    return nodes, edges, water_consumption

In [ ]:
base_dir = '/mnt/c/Users/rames102/Desktop/Qualifier_2024'  # relpace with correct directory in your system
city = "Apulia"
coords, edges, water_consumption = WDN_network_data(city, base_dir)

In [ ]:
def Construct_Graph(city):

    """Constructs a Water Distribution Network (WDN) graph for a given city.

    This function constructs a graph using the node coordinates and edges imported previously.
    It creates a NetworkX graph, adds nodes with their coordinates as attributes, and adds edges.

    Args:
        city (str): The name of the city for which to construct the graph. The function assumes
            that the node coordinates and edges for the specified city have been imported.

    Returns:
        nx.Graph: A NetworkX graph object representing the WDN. If the coordinates or edges are
            not available, returns None.

    Raises:
        ValueError: If the city name is not valid or if the necessary data is missing.

    Examples:
        >>> G = Construct_Graph('Apulian')
        >>> print(G.nodes(data=True))
        [('J1', {'pos': (10.0, 20.0)}), ('J2', {'pos': (15.0, 25.0)})]
    """

    if coords is None or edges is None:
        return None
    
    G = nx.Graph()

    # Defining Nodes in their coordinates
    for node, coord in coords.items():
        G.add_node(node, pos=coord)

    # Add edges
    G.add_edges_from(edges)
    
    return G

In [ ]:
G = Construct_Graph(city)
print(G)

In [ ]:
def centrality(G, water_consumption):

    """Calculates centrality metrics for a given city's WDN graph and water consumption data.

    This function computes various centrality metrics for the provided NetworkX graph and calculates
    vertex costs based on water consumption data and centrality measures.

    Args:
        G (nx.Graph): A NetworkX graph representing the water distribution network (WDN) for the city.
        water_consumption (dict): A dictionary where keys are node names and values are the water consumption
            at each node.

    Returns:
        tuple: A tuple containing two elements:
            - vertex_cost (dict): Dictionary where keys are node names and values are the computed costs
              for each vertex based on normalized demand and degree centrality.
            - edge_betweenness (dict): Dictionary of edge betweenness centrality values, where keys are
              tuples representing edges and values are the betweenness centrality scores for those edges.

    Raises:
        KeyError: If a node from the graph is not found in the water consumption data.

    Examples:
        >>> vertex_cost, edge_betweenness = centrality(G, water_consumption)
        >>> print(vertex_cost)
        {'J1': 2.5, 'J2': 3.0}
        >>> print(edge_betweenness)
        {(J1, J2): 0.1, (J2, J3): 0.2}
    """
    
    # Calculating edge betweenness centrality
    edge_betweenness = nx.edge_betweenness_centrality(G)

    # Calculate degree centrality
    degree_centrality = nx.degree_centrality(G)

    # Edge betweenness centrality as weights
    for (u, v, d) in G.edges(data=True):
        d['weight'] = edge_betweenness[(u, v)]

    # Define the weights C and D (Need to identify the right values)
    C = 1.0
    D = 1.0

    def compute_normalized_demand(water_consumption):        
        max_demand = max(water_consumption.items(), key=operator.itemgetter(1))[1]
        return {node: val / max_demand for node, val in water_consumption.items()}

    normalized_demand = compute_normalized_demand(water_consumption)

    # Debugging: print the node names in the graph
    print("Graph nodes:", list(G.nodes()))
    
    # Debugging: print the keys in normalized_demand
    print("Normalized demand keys:", list(normalized_demand.keys()))

    # Vertex cost function:
    vertex_cost = {}
    for node in G.nodes():
        f_i = normalized_demand.get(node, None)   # function of the water need at each node i
        if f_i is None:
            print(f"Node {node} not found in normalized_demand")
            continue
        g_i = degree_centrality[node] / (len(G.nodes()) - 1)    # node weights
        vertex_cost[node] = C * f_i + D * g_i
    
    return vertex_cost, edge_betweenness


In [ ]:
VC, EB = centrality(G, water_consumption)
print(VC)
print(EB)

In [ ]:
def plot_WDN(G):

    """Plots the graph of the city's Water Distribution Network (WDN).

    This function visualizes the WDN graph using NetworkX and Matplotlib. It plots nodes with their
    positions, edges, and labels for both nodes and edges. It also includes a legend and title for the plot.

    Args:
        G (nx.Graph): A NetworkX graph object representing the water distribution network (WDN) for the city.

    Returns:
        None

    Raises:
        ValueError: If the graph `G` is None or the position dictionary is empty.

    Examples:
        >>> plot_WDN(G)
    """

    if G is None:
        print(f"Graph could not be constructed for {city}.")
        return

    pos = nx.get_node_attributes(G, 'pos')
    
    if not pos:
        print("Error: Position dictionary is empty.")
        return

    print("Position Dictionary:", pos)

    # Create plot
    plt.figure(figsize=(24, 16))
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='skyblue')
    nx.draw_networkx_edges(G, pos, edge_color='black')

    # Round off edge and node labels to two decimals for the plot
    edge_labels_rounded = {k: round(v, 2) for k, v in EB.items()}
    node_labels_rounded = {k: round(v, 2) for k, v in VC.items()}

    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels_rounded, label_pos=0.5, font_size=6, verticalalignment='bottom')
    nx.draw_networkx_labels(G, pos, labels=node_labels_rounded, font_size=8, font_color='black')

    # Add legend
    blue_patch = Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=10, label='Potential nodes for sensor placement')
    plt.legend(handles=[blue_patch], loc='best', fontsize='24')

    # plt.title(f"Water Distribution Network - {city}", fontsize = 80)
    plt.show()

In [ ]:
plot_WDN(G)

The solution to this graph problem is explored using different reformulations of the problem.

First, we formulate the graph optimization problem as a Mixed Integer Program (MIP) (Speziali et al. (2021)):


$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} & \sum_{i \in V} c_{i}x_{i} + \sum_{(i, j) \in E} w_{ij}(1 - x_{j} - x_{i} + x_{i}x_{j}) \\
    \textrm{s.t.} & \sum_{i \in V} x_{i} = s \\
    & x_{i} \in \{0,1 \}^{n}
\end{array}
$$

Here, $c_{i}$ represents the cost of the $i^{th}$ node, $w_{ij}$ represents the weight corresponding to the edge between nodes $i$ and $j$, and $x_{i} \in \{0, 1\}^{n}$ is a binary decision variable that indicates whether a sensor is placed at the $i^{th}$ node. $s$ is the predefined total number of sensors.

Next, the Quadratic Unconstrained Binary Optimzation (QUBO) formulation of the graph problem is explored. Consider the MIP model from before:
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} & \sum_{i \in V} c_{i}x_{i} + \sum_{(i, j) \in E} w_{ij}(1 - x_{j} - x_{i} + x_{i}x_{j}) \\
    \textrm{s.t.} & \sum_{i \in V} x_{i} = s \\
    & x_{i} \in \{0,1 \}^{n}
\end{array}
$$
To implement the problem as a QUBO, the constraint should be lifted up into the objective function such that the problem becomes unconstrained. This is performed as follows:
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} & \sum_{i \in V} c_{i}x_{i} + \sum_{(i, j) \in E} (w_{ij}(1 - x_{j} - x_{i} + x_{i}x_{j})) + \rho(\sum_{i \in V} x_{i} - s)^2 \\
\end{array}
$$
where $\rho$ is a scalar penalty term. Further simplification leads to
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} & \sum_{i \in V} c_{i}x_{i} + \sum_{(i, j) \in E} w_{ij} - \sum_{(i, j) \in E} w_{ij}x_{j} - \sum_{(i, j) \in E} w_{ij}x_{i} + \sum_{(i, j) \in E} w_{ij}x_{i}x_{j} + \rho\sum_{i \in V} x_{i}^2 - 2\rho\sum_{i \in V} x_{i}s + \rho s^2 \\
\end{array}
$$
Since, $x_{i}$ is a binary variable, we can write $x_{i}$ as $x_{i}^2$. Taking this into account, the problem now becomes: 
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} & \sum_{i \in V} c_{i}x_{i}^2 + \sum_{(i, j) \in E}w_{ij} - \sum_{(i, j) \in E} w_{ij}x_{j}^2 - \sum_{(i, j) \in E} w_{ij}x_{i}^2 + \sum_{(i, j) \in E} w_{ij}x_{i}x_{j} + \rho\sum_{i \in V} x_{i}^2 - 2\rho\sum_{i \in V} x_{i}^2s + \rho s^2 \\
\end{array}
$$
Performing some algebraic manipulations, we have our QUBO problem.
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} \left( \sum_{i \in V} (c_i + \rho - 2\rho s - w_{ij})x_i^2 + \sum_{(i,j) \in E} w_{ij}x_i x_j + \rho s^2 + \sum_{(i,j) \in E}{w_{ij}} \right) \\
\end{array}
$$
where $s$ is the total number of sensors that is predefined.

In [ ]:
def create_pyomo_model(G): 

    """Initializes a Pyomo model for optimizing sensor placement in a city's Water Distribution Network (WDN).

    This function sets up a Pyomo model with nodes, edges, and associated parameters such as demand,
    vertex cost, and edge betweenness. It also defines the binary decision variables for sensor placement.

    Args:
        G (nx.Graph): A NetworkX graph object representing the water distribution network (WDN) for the city.

    Returns:
        pyo.ConcreteModel: A Pyomo ConcreteModel instance with sets, parameters, and decision variables defined.

    Raises:
        ValueError: If the graph `G` is None.

    Examples:
        >>> model = create_pyomo_model(G)
        >>> print(model)
    """

    if G is None:
        print(f"{city} does not exist.")
        return None

    # Create a pyomo model
    model = pyo.ConcreteModel()

    # Define pyomo sets for graph G
    nodes = list(G.nodes())
    edges = list(G.edges())
    model.nodes = pyo.Set(initialize=nodes)
    model.edges = pyo.Set(initialize=edges, dimen=2)

    # Parameters
    demand = water_consumption
    model.demand = pyo.Param(model.nodes, initialize=demand, mutable=True)
    model.c = pyo.Param(model.nodes, initialize=VC, mutable=True)  # vertex cost
    model.w = pyo.Param(model.edges, initialize=EB, mutable=True)  # edge_betweenness for edge weights

    # Binary decision variable
    model.x = pyo.Var(model.nodes, within=pyo.Binary)

    return model

In [ ]:
model = create_pyomo_model(G)

In [ ]:
# Mixed Integer Program model (MIP)
    
def MIQP(G, model_in, s):

    """Generates a Mixed Integer Programming (MIP) model for the given sensor placement problem.

    This function defines the objective function and constraints for the MIP model, aiming to optimize
    sensor placement in the city's Water Distribution Network (WDN). The objective function minimizes
    the total cost based on vertex costs and edge betweenness, while the constraint ensures that exactly
    `s` sensors are placed.

    Args:
        G (nx.Graph): A NetworkX graph object representing the water distribution network (WDN) for the city.
        model (pyo.ConcreteModel): A Pyomo ConcreteModel instance with sets and parameters defined.
        s (int): The number of sensors to be placed in the network.

    Returns:
        pyo.ConcreteModel: The updated Pyomo model with the objective function and constraints added.

    Raises:
        ValueError: If the model is None.

    Examples:
        >>> model = MIQP(G, model, 5)
        >>> print(model.obj)
    """

    if model_in is None:
        return None

    model = model_in.clone()
    
    # Define objective function
    def objective_rule_MIQP(model):
        return sum(model.c[i] * model.x[i] for i in model.nodes) + \
                sum(model.w[(i, j)] * (1 - model.x[i] - model.x[j] + model.x[i] * model.x[j]) for (i, j) in model.edges)

    model.obj = pyo.Objective(rule=objective_rule_MIQP, sense=pyo.minimize)

    # Constraints
    def sensor_constraint_rule_MIQP(model):
        return sum(model.x[i] for i in model.nodes) <= s

    model.sensor_constraint_MIQP = pyo.Constraint(rule=sensor_constraint_rule_MIQP)

    return model

In [ ]:
# Quadratic Unconstrained Binary Optimization model (QUBO)

def QUBO(G, model_in, s, rho):

    """Generates a Quadratic Unconstrained Binary Optimization (QUBO) model for the given problem.

    This function defines the objective function for the QUBO model, which aims to optimize sensor placement
    in the city's Water Distribution Network (WDN). The objective function includes terms for vertex costs,
    edge interactions, and a penalty for deviating from the specified number of sensors.

    Args:
        G (nx.Graph): A NetworkX graph object representing the water distribution network (WDN) for the city.
        model (pyo.ConcreteModel): A Pyomo ConcreteModel instance with sets and parameters defined.
        s (int): The number of sensors to be placed in the network.
        rho (float): Penalty parameter for the constraint that exactly `s` sensors must be placed.

    Returns:
        pyo.ConcreteModel: The updated Pyomo model with the QUBO objective function added.

    Raises:
        ValueError: If the model is None.

    Examples:
        >>> model = QUBO(G, model, 5, 0.1)
        >>> print(model.obj)
    """

    if model_in is None:
        return None

    model = model_in.clone()

    # # Removing previous objective function definition:
    # if hasattr(model, 'obj'):
    #     model.del_component(model.obj)
    # model.obj = pyo.Objective(rule=objective_rule_MIQP, sense=pyo.minimize)

    # # Removing previous constraint definition:
    # if hasattr(model, 'sensor_constraint'):
    #     model.del_component(model.sensor_constraint_MIQP)
    # model.sensor_constraint_MIQP = pyo.Constraint(rule=sensor_constraint_rule_MIQP)


    # Define objective function 
    def objective_rule_QUBO(model):
        term1 = sum(model.c[i] * model.x[i] for i in model.nodes)
        term2 = sum(model.w[(i, j)] * (1 - model.x[i]) * (1 - model.x[j]) for (i, j) in model.edges)
        term3 = rho * (sum(model.x[i] for i in model.nodes) - s) ** 2
        return term1 + term2 + term3

    model.obj = pyo.Objective(rule=objective_rule_QUBO, sense=pyo.minimize)

    return model

    # No constraints for QUBO

In [ ]:
# Coverage plot

solver = pyo.SolverFactory('gurobi', solver_io='python')
objective_value_list_MIP = []

def coverage(G, s, rho):
    """Computes and plots coverage metrics for the Water Distribution Network (WDN) using both MIP and QUBO formulations.

    This function iteratively solves the MIP and QUBO models for different numbers of sensors and records
    the objective values. It uses the Gurobi solver to find the optimal solutions and appends the results
    to lists for comparison.

    Args:
        G (nx.Graph): A NetworkX graph object representing the water distribution network (WDN) for the city.
        s (int): The maximum number of sensors to be placed.
        rho (float): Penalty parameter for the QUBO model.

    Returns:
        tuple: A tuple containing two lists:
            - objective_value_list_MIQP (list): A list of objective values obtained from the MIP formulation.
            - objective_value_list_QUBO (list): A list of objective values obtained from the QUBO formulation.

    Raises:
        ValueError: If the solver or model is not properly defined.

    Examples:
        >>> objective_value_list_MIQP, objective_value_list_QUBO = coverage(G, 5, 0.1)
        >>> print(objective_value_list_MIQP)
        [100, 95, 90]
        >>> print(objective_value_list_QUBO)
        [102, 96, 89]
    """

    # MIP
    for s in range(0, len(G.nodes())+1):
        model_MIQP = MIQP(G, model, s)
        # Solve MIP and append data to list for each iteration
        results_MIQP = solver.solve(model_MIQP, tee=True) # MIP formulation result
        objective_value_MIP = pyo.value(model_MIQP.obj)
        print("Objective value from MIP formulation: \n", objective_value_MIP)
        print("\n")
        objective_value_list_MIP.append(objective_value_MIP)

    return objective_value_list_MIP

In [ ]:
def plot_coverage(s, objective_value_list_MIP):
    """
    Plots the objective value against the number of sensors and marks the minimum objective value.

    Args:
        s (int): The maximum number of sensors to be placed.
        objective_value_list_MIP (list): A list of objective values obtained from the MIP formulation.

    Returns:
        None

    Example:
        >>> plot_coverage(5, [100, 95, 90, 85, 80])
    """
    # Find the minimum objective value and its corresponding sensor count
    min_obj_value = min(objective_value_list_MIP)
    min_index = objective_value_list_MIP.index(min_obj_value)
    
    plt.figure(figsize=(16, 16))
    plt.plot(range(len(objective_value_list_MIP)), objective_value_list_MIP, marker='o', linestyle='-', color='b', label='Objective Value')
    
    # Mark the minimum objective value
    plt.plot(min_index, min_obj_value, marker='x', markersize=10, color='r', label=f'Min Objective Value: {min_obj_value}')
    
    plt.title('Objective Value vs Number of Sensors')
    plt.xlabel('Number of Sensors (s)')
    plt.ylabel('Objective Value')
    plt.xticks(range(len(objective_value_list_MIP)))  # Ensures that the x-axis shows each integer sensor count
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
s = list(range(0, len(G.nodes())+1)) # List of feasible number of sensors that can be placed in the WDN
rho = np.sum(np.abs(list(VC.values()))) + 1 # Constant for penalty term for QUBO
objective_value_list_MIP = coverage(G, s, rho)
plot_coverage(s, objective_value_list_MIP)

In [ ]:
# Select optimal parameters from coverage plot results

# Get the number of sensors as the minimum value in the objective_value_list_MIP list
objective_value_list_MIP_min = min(objective_value_list_MIP)
s = objective_value_list_MIP.index(objective_value_list_MIP_min)
print(s)

# s = 15 # Optimal number of sensors
rho = np.sum(np.abs(list(VC.values()))) + 1 # Optimal constant for penalty term for QUBO

model_MIQP = MIQP(G, model, s)
model_QUBO = QUBO(G, model, s, rho)

In [ ]:
model_MIQP.pprint()

In [ ]:
# Solve
solver = pyo.SolverFactory('gurobi', solver_io='python')
results_MIQP = solver.solve(model_MIQP, tee=True) # MIP formulation result

In [ ]:
results_QUBO = solver.solve(model_QUBO, tee=True) # QUBO formulation result

In [ ]:
def sensor_placement_results(model_MIQP, model_QUBO):

    """Prints and returns sensor placement results for MIP and QUBO models.

    This function displays the sensor placement status for each node in both MIP and QUBO models. It provides
    a clear view of which nodes have sensors placed according to each optimization approach. It also returns
    dictionaries with sensor placement information for further analysis.

    Args:
        model_MIQP (pyo.ConcreteModel): The Pyomo model with MIP formulation, including sensor placement variables.
        model_QUBO (pyo.ConcreteModel): The Pyomo model with QUBO formulation, including sensor placement variables.

    Returns:
        tuple: A tuple containing two dictionaries:
            - sensor_placement_MIQP (dict): Dictionary with node names as keys and binary values (0 or 1) indicating
              whether a sensor is placed at each node according to the MIP model.
            - sensor_placement_QUBO (dict): Dictionary with node names as keys and binary values (0 or 1) indicating
              whether a sensor is placed at each node according to the QUBO model.

    Examples:
        >>> sensor_placement_MIQP, sensor_placement_QUBO = sensor_placement_results(model_MIQP, model_QUBO)
        >>> print(sensor_placement_MIQP)
        {'J1': 1, 'J2': 0, 'J3': 1}
        >>> print(sensor_placement_QUBO)
        {'J1': 1, 'J2': 1, 'J3': 0}
    """

    # Print results for MIP
    print("MIP results")
    for node in model_MIQP.nodes:
        print(f"Node {node}: Sensor placed = {pyo.value(model_MIQP.x[node])}")

    sensor_placement_MIQP = {node: pyo.value(model_MIQP.x[node]) for node in model_MIQP.nodes}

    # Print results for QUBO
    print("\nQUBO results")
    for node in model_QUBO.nodes:
        print(f"Node {node}: Sensor placed = {pyo.value(model_QUBO.x[node])}")

    sensor_placement_QUBO = {node: pyo.value(model_QUBO.x[node]) for node in model_QUBO.nodes}

    return sensor_placement_MIQP, sensor_placement_QUBO


In [ ]:
sensor_placement_MIQP, sensor_placement_QUBO = sensor_placement_results(model_MIQP, model_QUBO)

In [ ]:
def plot_sensor_placement(city, sensor_placement, title): # subplot_index, 

    """Plots a city's Water Distribution Network (WDN) and the optimal sensor placement.

    This function visualizes the WDN graph with nodes colored based on sensor placement: red for nodes with
    sensors and sky blue for nodes without. It uses Matplotlib to create the plot and adds a legend to distinguish
    between sensor and non-sensor nodes.

    Args:
        city (str): The name of the city for which the WDN is plotted.
        sensor_placement (dict): A dictionary indicating sensor placement at each node (1 for sensor, 0 for no sensor).
        subplot_index (int): The index of the subplot where the WDN will be plotted (1 or 2).
        title (str): The title of the plot.

    Returns:
        None

    Raises:
        ValueError: If the graph `G` is None.

    Examples:
        >>> plot_sensor_placement('Apulian', {'J1': 1, 'J2': 0, 'J3': 1}, 1, 'Optimal Sensor Placement')
    """

    if G is None:
        print(f"Graph could not be constructed for {city}.")
        return

    pos = nx.get_node_attributes(G, 'pos')

    # Create plot
    # plt.subplot(2, 1, subplot_index)
    plt.plot(1, 1)
    
    # Determine node colors based on sensor placement
    node_colors = ['red' if sensor_placement.get(node, 0) == 1 else 'skyblue' for node in G.nodes()]
    
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, edge_color='black')
    #nx.draw_networkx_labels(G, pos, font_size=6, font_color='black')

    # Round off edge and node labels to two decimals for the plot
    edge_labels_rounded = {k: round(v, 2) for k, v in EB.items()}
    node_labels_rounded = {k: round(v, 2) for k, v in VC.items()}

    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels_rounded, label_pos=0.5, font_size=6, verticalalignment='bottom')
    nx.draw_networkx_labels(G, pos, labels=node_labels_rounded, font_size=8, font_color='black')

    # Add legend
    red_patch = Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Sensor placed')
    blue_patch = Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=10, label='No sensor')
    plt.legend(handles=[red_patch, blue_patch], loc='best', fontsize='24')

    #plt.title(title, fontsize=80)

In [ ]:
def plot_comparison(city, sensor_placement_MIQP, save_path=None): # sensor_placement_QUBO,

    """Plots a city's Water Distribution Network (WDN) with optimal sensor placements from both MIP and QUBO formulations.

    This function generates a single figure with two subplots, showing the WDN with sensor placements obtained
    through the MIP and QUBO models. It utilizes the `plot_sensor_placement` function to visualize the results.

    Args:
        city (str): The name of the city for which the WDN is plotted.
        sensor_placement_MIQP (dict): A dictionary indicating sensor placement at each node from the MIP formulation.
        sensor_placement_QUBO (dict): A dictionary indicating sensor placement at each node from the QUBO formulation.

    Returns:
        None

    Examples:
        >>> plot_comparison('Apulian', {'J1': 1, 'J2': 0, 'J3': 1}, {'J1': 0, 'J2': 1, 'J3': 1})
    """

    # Create the 'plots' directory if it doesn't exist
    if save_path:
        save_dir = os.path.dirname(save_path)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    # Save plot
    if save_path:
        plt.savefig(save_path)

    plt.figure(figsize=(24, 16))

    plot_sensor_placement(city, sensor_placement_MIQP,  f'Optimal Sensor Placement - {city}')

    # plot_sensor_placement(city, sensor_placement_MIQP, 1, f'Sensor Placement - {city} - MIP formulation')
    # plot_sensor_placement(city, sensor_placement_QUBO, 2, f'Sensor Placement - {city} - QUBO formulation')

    # Save plot
    if save_path:
        plt.savefig(save_path)

    plt.show()

In [ ]:
save_path = os.path.join('plots', 'sensor_placement_comparison.png')
plot_comparison(city, sensor_placement_MIQP, save_path=save_path) # sensor_placement_QUBO, 

## Simulated Annealing:
First, we build the adjacency matrix $Q$ that is obtained from the QUBO model. Remember that the QUBO problem is formulated as follows: 
$$
\begin{array}{rl}
    \displaystyle%
    \min_{x} \left( \sum_{i \in V} (c_i + \rho - 2\rho s - \sum_{j \in E} w_{ij})x_i^2 + \sum_{(i,j) \in E} w_{ij}x_i x_j + \rho s^2 + \sum_{(i,j) \in E}{w_{ij}} \right) \\
\end{array}
$$
where $s$ is the total number of sensors that is predefined. And, We know that in general, a QUBO problem is defined as: 
$$
\min_{x \in \{0,1 \}^n} \sum_{(ij) \in E} Q_{ij}x_i x_j + \sum_{i \in V}Q_{ii}x_i + c_Q = \min_{x \in \{0,1 \}^n}  x^\top Q x + c_Q
$$
where we optimize over binary variables $x \in \{ 0,1 \}^n$, on a constrained graph $G(V,E)$ defined by an adjacency matrix $Q$. We also include an arbitrary offset  $c_Q$.


Comparing the above problems, it is evident that 
$$
\begin{array}{rl}
    \displaystyle%
    Q_{ii} = (c_{i} + \rho -2\rho s) - \sum_{j \in V}w_{ij} \\
    
    Q_{ij} = w_{ij} \\ \\

    c_{Q} = \rho s^2 + \sum_{(i,j) \in E}{w_{ij}}
\end{array}
$$\

In [ ]:
def build_Q_matrix(G, s):

    """Builds the Q matrix for the Quadratic Unconstrained Binary Optimization (QUBO) problem based on the given graph and model.

    This function constructs the Q adjacency matrix used in QUBO formulations, incorporating vertex costs, edge weights (edge betweenness), and constraints.

    Args:
        G (nx.Graph): The NetworkX graph object representing the water distribution network (WDN).
        model (pyo.ConcreteModel): The Pyomo model with the necessary parameters for QUBO.
        s (int): The number of sensors to be placed.


    Returns:
        tuple: A tuple containing:
            - Q (np.ndarray): The Q matrix for the QUBO problem.
            - cQ (float): The constant term in the QUBO objective function.

    Raises:
        ValueError: If the model is None.

    Examples:
        >>> Q, cQ = build_Q_matrix(G, model, 5)
        >>> print(Q)
        [[ 1.2 -0.5  0. ]
         [-0.5  1.5 -0.3]
         [ 0.  -0.3  1.1]]
        >>> print(cQ)
        2.5
    """

    G = Construct_Graph(city)

    num_nodes = len(G.nodes()) # stores the number of nodes in the graph G
    nodes = list(G.nodes())

    # Correctly create node_costs using node names
    vertex_cost = np.array([VC[node] for node in nodes])
    weight = EB

    # Initialize the constraint matrix A and vector b
    A = np.ones((1, num_nodes))
    print(A)
    b = np.array([s])
    print(b)
    
    # Calculate rho
    #rho = np.sum(np.abs(vertex_cost)) + 1
    rho = 6
    print(rho)
  
    # Add node costs to the diagonal
    Q = np.diag(vertex_cost)
    print(Q)
    print(vertex_cost)
    
    # # Add edge weights to the Q matrix
    total_weight = 0
    print(weight)
    for (i, j), w in weight.items():
        if nodes.index(i) != nodes.index(j):
            Q[nodes.index(i), nodes.index(j)] += w/2
            Q[nodes.index(j), nodes.index(i)] += w/2
        Q[nodes.index(i), nodes.index(i)] -= w
        Q[nodes.index(j), nodes.index(j)] -= w
        total_weight += w
        print(i, j, w)
    print(total_weight)
    print(Q)
    
    # Adjust diagonal with constraints:
    Q += rho*np.matmul(A.T,A)
    print(Q)
    Q -= rho*2*np.diag(np.matmul(b.T,A))
    
    # Calculate constant term cQ 
    cQ = rho * np.matmul(b.T, b) + total_weight
    
    return Q, cQ

In [ ]:
s = 153
Q, cQ = build_Q_matrix(G, s)
print("Q matrix:\n", Q , "\n")
print("cQ value:\n", cQ)

In [ ]:
G = nx.from_numpy_matrix(Q)
nx.draw(G, with_labels=True)

In [ ]:
# Binary Quadratic model (BQM)

def QUBO_dimod(Q, beta):

    """Creates a Binary Quadratic Model (BQM) from a Q matrix and an offset.

    This function constructs a BQM, which is used for solving quadratic optimization problems using binary variables.
    It uses the `dimod` library to create the model from the Q adjacency matrix and an offset value, preparing it for use with
    optimization solvers.

    Args:
        Q (np.ndarray): The Q matrix representing the quadratic terms in the optimization problem.
        beta (float): The offset value for the BQM.

    Returns:
        dimod.BinaryQuadraticModel: The Binary Quadratic Model constructed from the Q matrix and offset.

    Examples:
        >>> Q = np.array([[1, -1], [-1, 2]])
        >>> beta = 0.5
        >>> bqm = QUBO_dimod(Q, beta)
        >>> print(bqm)
        BinaryQuadraticModel({0: 1, 1: -1}, {(0, 1): -1}, 0.5, dimod.BINARY)
    """

    # Binary Quadratic Model
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset = beta) 
    
    return bqm

In [ ]:
bqm = QUBO_dimod(Q, beta=cQ) # returns bqm value
print(bqm)

# # Use ExactSolver to sample the BQM
# exactSampler = dimod.reference.samplers.ExactSolver()
# exactSamples = exactSampler.sample(bqm)

# # Print the sample results
# print("Exact Solver Samples:")
# print(exactSamples)

# # If you want to extract and process the samples:
# for sample, energy in exactSamples.data(['sample', 'energy']):
#     print(sample, "Energy:", energy)

In [ ]:
# Some useful functions to get plots
def plot_enumerate(results, title=None):

    plt.figure()

    energies = [datum.energy for datum in results.data(
        ['energy'], sorted_by='energy')]

    if results.vartype == 'Vartype.BINARY':
        samples = [''.join(c for c in str(datum.sample.values()).strip(
            ', ') if c.isdigit()) for datum in results.data(['sample'], sorted_by=None)]
        plt.xlabel('bitstring for solution')
    else:
        samples = np.arange(len(energies))
        plt.xlabel('solution')

    plt.bar(samples,energies)
    plt.xticks(rotation=90)
    plt.ylabel('Energy')
    plt.title(str(title))
    print("minimum energy:", min(energies))

def plot_samples(results, title=None):
    plt.figure()
    if results.vartype == 'Vartype.BINARY':
        samples = [''.join(c for c in str(datum.sample.values()).strip(
            ', ') if c.isdigit()) for datum in results.data(['sample'], sorted_by=None)]
        plt.xlabel('bitstring for solution')
    else:
        samples = np.arange(len(energies))
        plt.xlabel('solution')

    counts = Counter(samples)
    total = len(samples)
    for key in counts:
        counts[key] /= total
    df = pd.DataFrame.from_dict(counts, orient='index').sort_index()
    df.plot(kind='bar', legend=None)

    plt.xticks(rotation=80)
    plt.ylabel('Probabilities')
    plt.title(str(title))
    plt.show()
    print("minimum energy:", min(energies))


def plot_energies(results, title=None, skip=1):
    # skip parameter given to avoid putting all xlabels
    energies = results.data_vectors['energy']
    occurrences = results.data_vectors['num_occurrences']
    counts = Counter(energies)
    total = sum(occurrences)
    counts = {}
    for index, energy in enumerate(energies):
        if energy in counts.keys():
            counts[energy] += occurrences[index]
        else:
            counts[energy] = occurrences[index]
    for key in counts:
        counts[key] /= total
    df = pd.DataFrame.from_dict(counts, orient='index').sort_index()
    ax = df.plot(kind='bar', legend=None)

    plt.xlabel('Energy')
    plt.ylabel('Probabilities')
    # Plot only a subset of xlabels (every skip steps)
    ax.set_xticklabels([t if not i%skip else "" for i,t in enumerate(ax.get_xticklabels())])
    plt.title(str(title))
    plt.show()
    print("minimum energy:", min(energies))

In [ ]:
# Simulated Annealing 
mod = bqm
simAnnSampler = neal.SimulatedAnnealingSampler()
start_SA = time.time()
simAnnSamples = simAnnSampler.sample(mod, num_reads=4000)
end_SA = time.time()
total_time_SA = end_SA - start_SA
print("Total time for Simulated Annealing:", total_time_SA)
plot_enumerate(simAnnSamples, title='Simulated annealing in default parameters')
plot_energies(simAnnSamples, title='Simulated annealing in default parameters')
simAnnSamples.info

In [ ]:
# Tabu search
start_tabu = time.time()
sampleset_tabu = TabuSampler().sample(mod, num_reads=4000)
end_tabu = time.time()
total_time_tabu = end_tabu - start_tabu
print("Total time for Tabu search:", total_time_tabu)
plot_enumerate(sampleset_tabu, title='Tabu search in default parameters')
plot_energies(sampleset_tabu, title='Tabu search in default parameters')
sampleset_tabu.info

In [ ]:
# plot_enumerate(exactSamples, title='Enumerate all solutions')
# plot_energies(exactSamples, title='Enumerate all solutions', skip=10)

In [ ]:
print(simAnnSamples)
print(simAnnSamples.data)

In [ ]:
print(sampleset_tabu)
print(sampleset_tabu.data)

## Quantum Annealing via D-Wave:
This is the first interaction with D-Wave's Quantum Annealer. It will use the QUBO model introduced earlier and will define it using D-Wave's package dimod, and then solve them using neal's implementation of simulated annealing classicaly and D-Wave system package to use Quantum Annealing. We will also leverage the use of Networkx for network models/graphs.


In [ ]:
# Import DWave
import dwave_networkx as dnx
from dwave.system import (DWaveSampler, EmbeddingComposite,
                          FixedEmbeddingComposite)
from pprint import pprint
from dwave.system import LeapHybridSampler


In [ ]:
# Graph corresponding to D-Wave Model
qpu = DWaveSampler()
qpu_edges = qpu.edgelist
qpu_nodes = qpu.nodelist
# pprint(dir(qpu))
if qpu.solver.id == "DW_2000Q_6":
    print(qpu.solver.id)
    X = dnx.chimera_graph(16, node_list=qpu_nodes, edge_list=qpu_edges)
    dnx.draw_chimera(X, node_size=1)
    print('Number of qubits=', len(qpu_nodes))
    print('Number of couplers=', len(qpu_edges))
else:
    print(qpu.solver.id)
    X = dnx.pegasus_graph(16, node_list=qpu_nodes, edge_list=qpu_edges)
    dnx.draw_pegasus(X, node_size=1)
    print('Number of qubits=', len(qpu_nodes))
    print('Number of couplers=', len(qpu_edges))
     
DWavesampler = EmbeddingComposite(DWaveSampler())
DWaveSamples = DWavesampler.sample(bqm, num_reads=3500, 
                                   return_embedding=True, 
                                  #  chain_strength=chain_strength, 
                                  #  annealing_time=annealing_time
)
print(DWaveSamples.info)
embedding = DWaveSamples.info['embedding_context']['embedding']
if qpu.solver.id == "DW_2000Q_6":
  dnx.draw_chimera_embedding(X, embedding, node_size=2)
else:
  dnx.draw_pegasus_embedding(X, embedding, node_size=2)

In [ ]:
plot_enumerate(DWaveSamples, title='Quantum annealing in default parameters')
plot_energies(DWaveSamples, title='Quantum annealing in default parameters')

In [ ]:
print(DWaveSamples)
print(DWaveSamples.data)

In [ ]:
# Leap Hybrid Solver
sampler = LeapHybridSampler(solver={'category': 'hybrid'})
sampleset_Leap = sampler.sample(bqm)
sampleset_Leap.info  

In [ ]:
plot_enumerate(sampleset_Leap, title='hybrid quantum-classical in default parameters')
plot_energies(sampleset_Leap, title='hybrid quantum-classical in default parameters')

In [ ]:
def process_samples(simAnnSamples, DWaveSamples, sampleset_tabu, sampleset_Leap):
    """
    Processes D-Wave's Simulated and Quantum Annealing samplesets to calculate frequency, 
    mean energy, and probability for each energy level.

    This function converts D-Wave's Simulated and Quantum Annealing samplesets to pandas 
    DataFrames, aggregates the results by grouping by sample values, calculates the 
    frequency and mean energy for each group, and then calculates the probability for 
    each energy level. The resulting DataFrames are sorted by mean energy in ascending 
    order.

    Args:
        simAnnSamples: A sampleset object from which the D-Wave Simulated Annealing data will 
                       be converted into a pandas DataFrame. The sampleset is expected 
                       to contain 'energy' and 'num_occurrences' columns.
        DWaveSamples: A sampleset object from which the D-Wave Quantum Annealing data will be converted 
                      into a pandas DataFrame. The sampleset is expected to contain 
                      'energy' and 'num_occurrences' columns.
        TabuSamples: A sampleset object from which the Tabu search data will be converted
                        into a pandas DataFrame. The sampleset is expected to contain
                        'energy' and 'num_occurrences' columns.
        LeapSamples: A sampleset object from which the Leap Hybrid Solver data will be converted
                        into a pandas DataFrame. The sampleset is expected to contain
                        'energy' and 'num_occurrences' columns.

    Returns:
        tuple: A tuple containing two pandas DataFrames:
            - simAnn_df: DataFrame with mean energy and probability for Simulated Annealing.
            - dwave_df: DataFrame with mean energy and probability for D-Wave.
            - tabu_df: DataFrame with mean energy and probability for Tabu search.
            - leap_df: DataFrame with mean energy and probability for Leap Hybrid Solver.

    Example:
        simAnn_df, dwave_df,  = process_samples(simAnnSamples, DWaveSamples)
    """
    # Convert Simulated Annealing sampleset to pandas dataframe
    SimAnn_Samples_df = simAnnSamples.to_pandas_dataframe()

    # Aggregate the results for Simulated Annealing
    grouped_simAnn = SimAnn_Samples_df.groupby(SimAnn_Samples_df.columns.difference(['energy', 'num_occurrences']).tolist())
    result_simAnn = grouped_simAnn.agg(
        frequency=('num_occurrences', 'sum'),
        mean_energy=('energy', 'mean')
    ).reset_index()

    # Calculate the total number of Simulated Annealing samples
    total_samples_simAnn = result_simAnn['frequency'].sum()

    # Calculate the probability for each energy level in Simulated Annealing
    result_simAnn['probability'] = result_simAnn['frequency'] / total_samples_simAnn

    # Sort the Simulated Annealing results by mean_energy in ascending order
    simAnn_df = result_simAnn.sort_values(by='mean_energy')

    # Convert D-Wave sampleset to pandas dataframe
    DWave_Samples_df = DWaveSamples.to_pandas_dataframe()

    # Aggregate the results for D-Wave
    grouped_DWave = DWave_Samples_df.groupby(DWave_Samples_df.columns.difference(['energy', 'num_occurrences']).tolist())
    result_DWave = grouped_DWave.agg(
        frequency=('num_occurrences', 'sum'),
        mean_energy=('energy', 'mean')
    ).reset_index()

    # Calculate the total number of D-Wave samples
    total_samples_DWave = result_DWave['frequency'].sum()

    # Calculate the probability for each energy level in D-Wave
    result_DWave['probability'] = result_DWave['frequency'] / total_samples_DWave

    # Sort the D-Wave results by mean_energy in ascending order
    dwave_df = result_DWave.sort_values(by='mean_energy')
    
    # Convert tabu search sampleset to pandas dataframe
    tabu_Samples_df = sampleset_tabu.to_pandas_dataframe()

    # Aggregate the results for tabu search
    grouped_tabu = tabu_Samples_df.groupby(tabu_Samples_df.columns.difference(['energy', 'num_occurrences']).tolist())
    result_tabu = grouped_tabu.agg(
        frequency=('num_occurrences', 'sum'),
        mean_energy=('energy', 'mean')
    ).reset_index()

    # Calculate the total number of tabu search samples
    total_samples_tabu = result_tabu['frequency'].sum()

    # Calculate the probability for each energy level in tabu search
    result_tabu['probability'] = result_tabu['frequency'] / total_samples_tabu

    # Sort the tabu search results by mean_energy in ascending order
    tabu_df = result_tabu.sort_values(by='mean_energy')

    # Convert Leap Hybrid Solver sampleset to pandas dataframe
    leap_Samples_df = sampleset_Leap.to_pandas_dataframe()

    # Aggregate the results for Leap Hybrid Solver
    grouped_leap = leap_Samples_df.groupby(leap_Samples_df.columns.difference(['energy', 'num_occurrences']).tolist())
    result_leap = grouped_leap.agg(
        frequency=('num_occurrences', 'sum'),
        mean_energy=('energy', 'mean')
    ).reset_index()

    # Calculate the total number of Leap Hybrid Solver samples
    total_samples_leap = result_leap['frequency'].sum()

    # Calculate the probability for each energy level in Leap Hybrid Solver
    result_leap['probability'] = result_leap['frequency'] / total_samples_leap

    # Sort the Leap Hybrid Solver results by mean_energy in ascending order
    leap_df = result_leap.sort_values(by='mean_energy')

    # Print the dataframes
    pd.set_option('display.max_rows', None)  # Ensure all rows are printed
    print("Tabu Search Results:")
    print(tabu_df[['mean_energy', 'probability']])
    print("\nLeap Hybrid Solver Results:")
    print(leap_df[['mean_energy', 'probability']])

    return simAnn_df, dwave_df, tabu_df, leap_df

In [ ]:
results = process_samples(simAnnSamples, DWaveSamples, sampleset_tabu, sampleset_Leap)
simAnn_df, dwave_df, tabu_df, leap_df = results

In [ ]:
def plot_energy_probability(dwave_df, simAnn_df, tabu_df, leap_df, num_samples=10, width=0.3, line_x=4.871023058796, y_max=0.002):
    df_list = [dwave_df, simAnn_df, tabu_df, leap_df]  # Updated list of dataframes
    df_names = ['Quantum Annealing', 'Simulated Annealing', 'Tabu Search', 'Leap Hybrid Solver']  # Updated names
    df_colors = ['blue', 'red', 'yellow', 'purple']  # Updated colors
    
    # Filter each dataframe to include only the first `num_samples` samples
    filtered_dwave_df = dwave_df.head(num_samples)
    filtered_simAnn_df = simAnn_df.head(num_samples)
    filtered_tabu_df = tabu_df.head(num_samples)
    filtered_leap_df = leap_df.head(num_samples)

    # Round the mean_energy values to 4 decimal places
    filtered_dwave_df['mean_energy'] = filtered_dwave_df['mean_energy'].round(4)
    filtered_simAnn_df['mean_energy'] = filtered_simAnn_df['mean_energy'].round(4)
    filtered_tabu_df['mean_energy'] = filtered_tabu_df['mean_energy'].round(4)
    filtered_leap_df['mean_energy'] = filtered_leap_df['mean_energy'].round(4)

    # Define the x-axis range from 2 to 3.2
    x_range = np.arange(3, 10)

    # Combine x_range with specific values from the data
    x_labels = sorted(set(filtered_dwave_df['mean_energy']).union(set(filtered_simAnn_df['mean_energy'])).union(set(filtered_tabu_df['mean_energy'])).union(set(filtered_leap_df['mean_energy'])).union(set(x_range)))

    # Calculate positions for bars for both datasets
    pos_dwave = [x_labels.index(val) for val in filtered_dwave_df['mean_energy']]
    pos_simAnn = [x_labels.index(val) for val in filtered_simAnn_df['mean_energy']]
    pos_tabu = [x_labels.index(val) for val in filtered_tabu_df['mean_energy']]
    pos_leap = [x_labels.index(val) for val in filtered_leap_df['mean_energy']]

    # Create a bar plot
    fig, ax = plt.subplots(figsize=(12, 6))

    # Create bars for dwave_df data
    plt.bar(np.array(pos_dwave) - width / 2, 
            filtered_dwave_df['probability'], 
            width, 
            color=df_colors[0], 
            label=df_names[0], 
            align='center', 
            alpha=0.7)  # Adjust transparency for better visibility

    # Create bars for simAnn_df data
    plt.bar(np.array(pos_simAnn) + width / 2, 
            filtered_simAnn_df['probability'],
            width, 
            color=df_colors[1], 
            label=df_names[1], 
            align='center', 
            alpha=0.7)  # Adjust transparency for better visibility

    # Create bars for tabu_df data
    plt.bar(np.array(pos_tabu) + width / 2,
            filtered_tabu_df['probability'],
            width,
            color=df_colors[2],
            label=df_names[2],
            align='center',
            alpha=0.7)  # Adjust transparency for better visibility

    # Create bars for leap_df data
    plt.bar(np.array(pos_leap) + width / 2,
            filtered_leap_df['probability'],
            width,
            color=df_colors[3],
            label=df_names[3],
            align='center',
            alpha=0.7)  # Adjust transparency for better visibility
            
    # Add a dashed vertical line at objective vaue obtained through MIP formulation
    if line_x in x_labels:
        ax.axvline(x=x_labels.index(line_x), color='green', linestyle='--', label='Gurobi')
    else:
        # If the exact value is not in x_labels, use the closest index
        nearest_idx = min(range(len(x_labels)), key=lambda i: abs(x_labels[i] - line_x))
        ax.axvline(x=nearest_idx, color='green', linestyle='--', label='Gurobi')

    # Set the y-axis label
    ax.set_ylabel('Probability')
    ax.set_xlabel('Mean Energy')

    # Set the chart's title
    ax.set_title(f'Energy vs Probability for Different Solvers - {city} WDN')

    # Set the x-axis limits to ensure the vertical line is visible
    ax.set_xlim(-0.5, len(x_labels) - 0.5)

    # Set the y-axis limits to accommodate the desired range
    ax.set_ylim(0, y_max)  # Adjust the upper limit as needed

    # Set the x-axis ticks and labels
    ax.set_xticks(np.arange(len(x_labels)))
    ax.set_xticklabels([f"{x_labels[i]:.4f}" for i in np.arange(len(x_labels))], rotation=45, ha='right')

    # Display the legend
    plt.legend(loc='upper right')

    plt.show()

In [ ]:
plot_energy_probability(dwave_df, simAnn_df, tabu_df, leap_df)

## Time To Solution (TTS):
We would like to obtain a large probability of finding a right solution (the definition of right comes from what you define as success). On the other hand, the time it takes to solve these cases should be as small as possible. Therefore we are interested in a metric that combines both, and is called the Time To Solution (TTS) which is defined as
$$
TTS = T_{single run} * \frac{\log({1-s})}{\log({1-p})}
$$
where s is a success factor, and taken as $s = 99\%$, while $p$ is the success probability, usually accounted as the observed success probability.

One usually reads this as the time to solution within 99\% probability.


In [ ]:
# def calculate_tts(samples, s, min_energy, TT):
#     """
#     Calculates the Time-to-Solution (TTS) for a given set of samples.

#     Args:
#         samples (iterable): A collection of samples, each containing a 'sample', 
#                             'energy', and 'num_occurrences'.
#         s (int): The expected total size for feasible solutions.
#         energy_threshold (float): The energy threshold below which a solution is 
#                                   considered optimal.
#         total_time (float): The total time used for calculations, typically representing 
#                             the time taken to run the optimization process.

#     Returns:
#         tuple: A tuple containing the following:
#             - p_opt (float): The probability of finding an optimal solution.
#             - p_fea (float): The probability of finding a feasible solution.
#             - TTS_opt (float): The Time-to-Solution for finding an optimal solution.
#             - TTS_fea (float): The Time-to-Solution for finding a feasible solution.

#     Example:
#         tts_results = calculate_tts(simAnnSamples.data(['sample', 'energy', 'num_occurrences']), 10, 2.68, total_time)
#     """
#     p_opt = 0
#     p_fea = 0
    
#     # Convert the generator to a list if needed
#     samples_data = list(samples)
#     total_samples = len(samples_data)
    
#     # Calculate total occurrences for feasible and optimal samples
#     for sample, energy, num_ocu in samples_data:
#         total_size = sum(sample.values())
#         if total_size == s:
#             print(total_size)
#             p_fea += num_ocu
#             if energy <= min_energy + 1e-6:
#                 p_opt += num_ocu

#     # Normalize by the total number of samples
#     p_opt /= total_samples
#     p_fea /= total_samples

#     # Calculate TTS values
#     TTS_opt = TT * np.log(1 - 0.99) / np.log(1 - p_opt)
#     TTS_fea = TT * np.log(1 - 0.99) / np.log(1 - p_fea)

#     return p_opt, p_fea, TTS_opt, TTS_fea

In [ ]:
def calculate_tts(sample_set, s, energy_threshold, exec_time): 
    """
    Calculate the time to solution (optimality or feasibility) for the simulated annealing solver.

    Parameters
    ----------
    dist_sim : dimod.SampleSet
        The samples returned by the simulated or quantum annealing solver.    
    exec_time : float
        The execution time of the simulated or quantum annealing solver.
    energy_threshold : float
        The energy threshold for the optimal or feasible solution.
    s : float, optional
        The success probability, by default 0.99
    """

    # dist = sample_set.aggregate()
    samples_data = list(sample_set)
    tts = exec_time
    total_size = s

    # Define probability of finding solution in dist with energy <= energy_threshold 
    p_opt = 0
    p_fea = 0
    n = 0 # total number of samples

    for sample, energy, num_ocu in samples_data:
        #print(sample, energy, num_ocu)
        total_size = sum(sample.values())
        print(total_size)
        if total_size == s:
            p_fea += num_ocu
        if energy <= energy_threshold:
            p_opt += num_ocu

        n += num_ocu
        
    # Normalize the probability
    p_opt = p_opt/n
    p_fea = p_fea/n
    print(f'Probability of optimal solution <= {energy_threshold} : {p_opt}')
    print(f'Probability of feasible solution sataisfying constraints : {p_fea}')

    # Compute time to solution for optimal solution
    if p_opt == 1:
        TTS_opt = tts
    elif p_opt == 0:
        TTS_opt = np.inf
    else:
        TTS_opt = tts*np.log(1 - 0.99)/np.log(1 - p_opt)

    # Compute time to solution for feasible solution
    if p_fea == 1:
        TTS_fea = tts
    elif p_fea == 0:
        TTS_fea = np.inf
    else:
        TTS_fea = tts*np.log(1 - 0.99)/np.log(1 - p_fea)

    print(f'Time to solution for optimal solution : {TTS_opt}')
    print(f'Time to solution for feasible solution : {TTS_fea}')

In [ ]:
# TTS for Simulated Annealing samples
tts_results_SA = calculate_tts(simAnnSamples.data(['sample', 'energy', 'num_occurrences']), 153, 28.25, total_time_SA)
print(tts_results_SA)

In [ ]:
# TTS for Tabu Search samples
tts_results_Tabu = calculate_tts(sampleset_tabu.data(['sample', 'energy', 'num_occurrences']), 153, 12.154, total_time_tabu)
print(tts_results_Tabu)

In [ ]:
# TTS for Leap Hybrid Solver samples
total_time_Leap = sampleset_Leap.info['run_time']
print(total_time_Leap)
tts_results_Leap = calculate_tts(sampleset_Leap.data(['sample', 'energy', 'num_occurrences']), 153, 22.841, total_time_Leap)
print(tts_results_Leap)

In [ ]:
# TTS for Quantum Annealing samples
# Assign the correct key for the total time to total_time_QA
total_time_QA = DWaveSamples.info['timing']['qpu_access_time']
print(total_time_QA)
tts_results_QA = calculate_tts(DWaveSamples.data(['sample', 'energy', 'num_occurrences']), 15, 610, total_time_QA)
print(tts_results_QA)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Provided data
nodes = [24, 37, 114, 272, 856]
gurobi_tts_MIP = [0.07, 0.06, 0.08, 0.03, 0.11]
sim_ann_tts = [81.60481582, 96.50665403, 440010.0127, 2192982.025, 23942409.5]
tabu_search_tts = [84.32313108, 84.47434545, 10733.85075, 5750.226611, 253908.6454]
leap_hybrid_tts = [2.98621, 2.980085, 2.998373, 3.000249, 2.997319]
quant_ann_tts = [8193.967296, 2869.983265, 13853.31548, np.inf, np.inf]  # Using np.inf for 'Infinity'

# Define a high value to plot infinities
high_value = max(max(gurobi_tts_MIP), max(sim_ann_tts), max(tabu_search_tts), max(leap_hybrid_tts)) * 10

# Plotting
plt.figure(figsize=(24, 10))
plt.plot(nodes, gurobi_tts_MIP, 'o-', label='Gurobi', color='green')
plt.plot(nodes, sim_ann_tts, 's-', label='Simulated Annealing', color='red')
plt.plot(nodes, tabu_search_tts, 'd-', label='Tabu Search', color='yellow')
plt.plot(nodes, leap_hybrid_tts, 'x-', label='Leap Hybrid', color='maroon')

# Plot Quantum Annealing with high value for infinity
plt.plot(nodes, [high_value if x == np.inf else x for x in quant_ann_tts], 'v-', label='Quantum Annealing', color='blue')

# Add annotations for infinity values
for i, value in enumerate(quant_ann_tts):
    if value == np.inf:
        plt.text(nodes[i], high_value, '∞', color='blue', ha='center', va='bottom')

# Add dotted vertical lines and labels for each node
for node in nodes:
    plt.axvline(x=node, color='gray', linestyle='--', linewidth=0.5)

# Customizing the x-axis ticks and labels
plt.xlabel('Node Size of WDN', fontsize='20')
plt.ylabel('TTS for Best Solution (s)', fontsize='20')
# plt.xticks(fontsize=16)
plt.xticks(nodes, fontsize=16)  # Set x-axis ticks to the exact node sizes with font size 12
plt.yticks(fontsize=16)  # Set y-axis ticks font size to 12
plt.legend(fontsize=16)  # Increase the font size of the legend
plt.yscale('log')  # Using logarithmic scale due to large range of TTS values
plt.ylim(0, high_value * 1.1)  # Extend the y-axis limit to accommodate the high values
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.show()
